### Vamos Generalizar a otimização da carteira

Vamos ver como se sai o modelo com ativos norte americanos, e a ideia é nós aplicarmos posteriormente com ativos do mercado brasileiro.
Vou selecionar 4 ações, a ideia que eu tenho é que essas ações são potencialmente rentáveis, porém não sei quanto investir em cada uma delas e como mitigar os meus riscos.


### Distribuição dos retorno

Retorno líquido: 
$$Retorno = R = \dfrac{P_{t}}{P_{t-1}} - 1$$

Retorno "Bruto": 
$$R+1 = \dfrac{P_{t}}{P_{t-1}} $$

## Sharpe Ratio

$$SR(w) = \dfrac{R(w) - R_f}{\sigma(w)}$$

Em que:
$$
w = \begin{bmatrix}
w_1 & w_2 & w_3 & w_4
\end{bmatrix}
$$

$$\sum_{i} ^{4} w_i = 1$$

é o vetor dos pesos normalizados.

- $SR(w)$: Sharpe ratio
- $R(f)$ : taxa livre de risco
- $R(w)$ : retorno do portfolio

$$R(w)  = w^{T} \mathbf{R} $$
$$\sigma(w) = \sqrt(w^{T}\Sigma w)$$

- $\Sigma$ é a matriz de covariância

Perceba que o máximo Sharpe ratio é tal que é proporcional à
$$SR(w) = \dfrac{\mathbf{r}}{\sqrt(w^{T}\Sigma w)}$$

In [25]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf


tickers = ['AAPL' , 'GOOGL', 'BRK-B' , 'PFE.DE','GC=F'] #definimos os tickers das ações 
## Ações 
prices = yf.download(tickers=tickers , start = '2017-01-01', end='2020-01-01')['Adj Close']
prices.dropna(inplace=True)


returns = prices.pct_change().dropna()
#returns = (prices/prices.shift(1) -1)
returns.dropna(inplace=True)
ativos = returns.columns.to_list()
returns


[*********************100%%**********************]  5 of 5 completed


Ticker,AAPL,BRK-B,GC=F,GOOGL,PFE.DE
Date,,,,,
2017-01-04,-0.001119,0.001526,0.002930,-0.000297,0.000474
2017-01-05,0.005085,-0.004754,0.013662,0.006499,-0.004895
2017-01-06,0.011149,0.000674,-0.006612,0.014994,0.007299
2017-01-09,0.009159,-0.008506,0.009898,0.002387,-0.001260
2017-01-10,0.001008,-0.003395,0.000591,-0.001414,-0.004259
...,...,...,...,...,...
2019-12-19,0.001001,0.000889,0.003803,0.003351,0.005747
2019-12-20,-0.002071,0.004884,-0.002368,-0.003848,0.020000
2019-12-23,0.016319,-0.003668,0.005289,-0.000437,-0.008403


In [26]:
import numpy as np
import pandas as pd

# Assumindo que 'returns' é o DataFrame com os retornos diários das ações
# e que 'returns' já foi carregado anteriormente

number_sim = 1000
num_assets = len(tickers)  # Número de ativos
weight = np.zeros((number_sim, num_assets))
Sigma = returns.cov() # matriz de covariancia
mean_returns = returns.mean()

expected_return = np.zeros(number_sim)
expected_risk = np.zeros(number_sim)
sharpe_ratio = np.zeros(number_sim)

for k in range(number_sim):
    # Gerar pesos aleatórios e normalizados
    w = np.random.random(num_assets)
    w = w / np.sum(w)
    weight[k, :] = w 

    # Retorno  esperado (anualizado)
    expected_return[k] = np.sum(mean_returns * w) * 252

    # Risco (anualizado)
    expected_risk[k] = np.sqrt(np.dot(w.T, np.dot(Sigma, w)) * 252)

    # Índice de Sharpe
    sharpe_ratio[k] = (expected_return[k] / expected_risk[k]) 

# Criar DataFrame
portfolio_df = pd.DataFrame(weight, columns=[f'{tickers[i]}' for i in range(num_assets)])
portfolio_df['Expected Return(%)'] = expected_return*100
portfolio_df['Risk(%)'] = expected_risk *100
portfolio_df['Sharpe Ratio'] = sharpe_ratio



portfolio_df


,AAPL,GOOGL,BRK-B,PFE.DE,GC=F,Expected Return(%),Risk(%),Sharpe Ratio
0,0.285753,0.188288,0.221765,0.092097,0.212097,18.706460,11.579740,1.615447
1,0.045458,0.395564,0.170842,0.004758,0.383378,11.982034,10.964296,1.092823
2,0.282610,0.090742,0.247166,0.229588,0.149894,19.785000,12.475922,1.585855
3,0.043917,0.276704,0.090894,0.228282,0.360203,13.925759,12.132621,1.147795
4,0.193769,0.166092,0.132013,0.265002,0.243124,17.988029,12.774222,1.408151
...,...,...,...,...,...,...,...,...
995,0.174711,0.127553,0.276488,0.282243,0.139005,17.572013,11.638861,1.509771
996,0.398154,0.306909,0.089122,0.045732,0.160082,21.528542,14.496248,1.485111
997,0.028532,0.152898,0.145411,0.330488,0.342672,14.240831,11.981210,1.188597
998,0.153564,0.087329,0.074170,0.219193,0.465744,16.211388,12.872179,1.259413


In [27]:
import plotly.express as px

# Encontrar a linha com o máximo índice de Sharpe
max_sharpe_idx = portfolio_df['Sharpe Ratio'].idxmax()
max_sharpe_portfolio = portfolio_df.loc[max_sharpe_idx]

# Encontrar a linha com a mínima variância (risco)
min_variance_idx = portfolio_df['Risk(%)'].idxmin()
min_variance_portfolio = portfolio_df.loc[min_variance_idx]

# Automatizar a criação do dicionário hover_data
hover_data = {col: ':.4f' for col in portfolio_df.columns}

# Criar gráfico de dispersão com Plotly, incluindo os pesos no hover_data
fig = px.scatter(
    portfolio_df, 
    x='Risk(%)', 
    y='Expected Return(%)', 
    color='Sharpe Ratio',
    title='Risco vs Retorno com Índice de Sharpe por  Monte Carlo',
    labels={'Risk(%)': 'Risco (%)', 'Expected Return(%)': 'Retorno (%)'},
    color_continuous_scale=px.colors.sequential.Bluered,
    hover_data=hover_data
)

# Adicionar contorno aos marcadores de dispersão
fig.update_traces(marker=dict(line=dict(width=2, color='black')))

# Adicionar marcadores para o portfólio com máximo Sharpe e mínima variância
fig.add_scatter(
    x=[max_sharpe_portfolio['Risk(%)']], 
    y=[max_sharpe_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='green', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Máximo Sharpe',
)

fig.add_scatter(
    x=[min_variance_portfolio['Risk(%)']], 
    y=[min_variance_portfolio['Expected Return(%)']], 
    mode='markers+text', 
    marker=dict(color='blue', size=12, symbol='star', line=dict(color='black', width=1)), 
    name='Mínima Variância'
)

# Atualizar layout para mover a legenda para a parte inferior
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

# Mostrar gráfico
fig.show()


In [28]:
correlation_matrix = round(returns.corr(), 3)
# Mascarar a matriz de correlação para obter apenas a triangular inferior
mask = np.tril(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.mask(~mask)

# Plotar a matriz de correlação usando Plotly
fig = px.imshow(
    correlation_matrix_masked, 
    text_auto=True, 
    aspect="equal", 
    color_continuous_scale='Blues',  # Escala de cor azul
    title="Matriz de Correlação"
)

# Melhorar o aspecto do gráfico
fig.update_layout(
    width=600,  # Largura da figura
    height=600,  # Altura da figura
    margin=dict(l=100, r=50, t=50, b=50),  # Margens
    font=dict(size=14),  # Tamanho da fonte
    plot_bgcolor='white',  # Cor de fundo branca
    paper_bgcolor='white'  # Cor de fundo branca
)

fig.show()

### Vamos analisar em termos de retornos como a carteira se sairia daqui 

In [29]:
min_variance_portfolio

AAPL                   0.051049
GOOGL                  0.255449
BRK-B                  0.588478
PFE.DE                 0.022663
GC=F                   0.082361
Expected Return(%)    11.979580
Risk(%)                7.837473
Sharpe Ratio           1.528500
Name: 147, dtype: float64

In [30]:
max_sharpe_portfolio

AAPL                   0.276023
GOOGL                  0.111410
BRK-B                  0.409894
PFE.DE                 0.019816
GC=F                   0.182856
Expected Return(%)    17.503164
Risk(%)                9.557129
Sharpe Ratio           1.831425
Name: 151, dtype: float64

In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

tickers.append('SPY')

# Criar o dicionário de pesos automaticamente a partir dos tickers
weights = {ticker: max_sharpe_portfolio[ticker] for ticker in tickers if ticker in max_sharpe_portfolio.index}

# Baixar os preços ajustados das ações e do índice SPY
prices = yf.download(tickers=tickers, start='2020-01-01', end='2022-01-01')['Adj Close']
prices.dropna(inplace=True)

# Calcular os retornos diários
returns = prices.pct_change().dropna()

# Calcular o retorno diário do portfólio
portfolio_returns = returns[list(weights.keys())].mul(list(weights.values()), axis=1).sum(axis=1)

# Calcular o retorno acumulado do portfóliocumprod(
cumulative_portfolio_returns = ((1 + portfolio_returns).cumprod() - 1)*100

# Calcular o retorno acumulado do SPY
cumulative_spy_returns = ((1 + returns['SPY']).cumprod() - 1)*100

# Plotar os retornos acumulados
fig = go.Figure()

fig.add_trace(go.Scatter(x=cumulative_portfolio_returns.index, y=cumulative_portfolio_returns, 
                         mode='lines', name='Portfólio', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=cumulative_spy_returns.index, y=cumulative_spy_returns, 
                         mode='lines', name='SPY', line=dict(color='red')))

fig.update_layout(title='Retorno Acumulado do Portfólio vs SPY',
                  xaxis_title='Data',
                  yaxis_title='Retorno Acumulado(%)',
                  template='plotly_white')

fig.show()

[*********************100%%**********************]  6 of 6 completed
